# Equal-Weight S&P 500
This project takes as input the value of a portfolio and outputs the number of shares of each S&P500 constituent that should be purchased to achieve an equal-weight version of the S&P500. 

### Imports

In [1]:
import pandas as pd
import requests
import math
import kagglehub as kh
from my_secrets import POLYGON_API_TOKEN
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
def get_most_recent_trading_day(date):
    while True:
        # Check if it's a weekend (Saturday or Sunday)
        if date.weekday() in [5, 6]:  # Saturday = 5, Sunday = 6
            date -= timedelta(days=1)
        else:
            # Make a dummy API call to check if the date is a holiday
            test_api_url = f"https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/{date.strftime('%Y-%m-%d')}/{date.strftime('%Y-%m-%d')}?apiKey={POLYGON_API_TOKEN}"
            response = requests.get(test_api_url).json()
            if 'results' in response and len(response['results']) > 0:
                break  # Valid trading day
            else:
                date -= timedelta(days=1)  # Move to the previous day if it's a holiday
    return date

In [3]:
path = kh.dataset_download("andrewmvd/sp-500-stocks")
stocks = pd.read_csv(path + "/sp500_companies.csv")
stocks = stocks['Symbol']
stocks

0       AAPL
1       NVDA
2       MSFT
3       AMZN
4      GOOGL
       ...  
497      CZR
498      BWA
499     QRVO
500      FMC
501     AMTM
Name: Symbol, Length: 502, dtype: object

In [4]:
# Define columns and initialize the DataFrame
cols = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=cols)
stocks_subset = stocks
data_rows = []

# Get today's date and adjust for trading days
today = datetime.today()
trading_date = get_most_recent_trading_day(today)
print(f"Getting data for {trading_date.strftime('%Y-%m-%d')}")
for symbol in tqdm(stocks_subset, desc="Processing Stocks"):
    date = trading_date.strftime('%Y-%m-%d')
    api_url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{date}/{date}?apiKey={POLYGON_API_TOKEN}"
    response = requests.get(api_url).json()
    #print(response)
    # Ensure the response has the expected structure
    if 'results' in response and len(response['results']) > 0:
        price = response['results'][0]['c']
        market_cap = price * response['results'][0]['v']
        data_rows.append(
            {
                'Ticker': symbol,
                'Stock Price': price,
                'Market Capitalization': market_cap,
                'Number of Shares to Buy': 'N/A'
            }
        )

# Create the DataFrame from the list of dictionaries
final_dataframe = pd.DataFrame(data_rows, columns=cols)

Getting data for 2025-01-10


Processing Stocks: 100%|██████████| 502/502 [02:07<00:00,  3.93it/s]


## Calculate the Number of Shares to Buy


In [ ]:
""" portfolio_size = input("Enter the value of your portfolio: ")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input("Enter the value of your portfolio: ")
    val = float(portfolio_size) """

portfolio_size = 1000000

positiion_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(positiion_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe.to_csv(f'recommended_positions_{trading_date}.csv', index=False)
#final_dataframe.to_csv(f'final_dataframe_{trading_date}.csv', index=False)
final_dataframe